In [ ]:
data_neta = pd.read_csv("./housing_BCN/data/base_dades_neta.csv")

In [ ]:
data_neta["neighborhood"] = data_neta["neighborhood"].replace({
    "Sarria-Sant Gervasi": "Sarrià-Sant Gervasi",
    "Horta- Guinardo": "Horta-Guinardó"})

In [ ]:
geojson_path = "./housing_BCN/mapas/distritos_barcelona.geojson"

barrios_gdf = gpd.read_file(geojson_path)

data_neta["neighborhood"] = data_neta["neighborhood"].str.strip()
barrios_gdf["nom_districte"] = barrios_gdf["nom_districte"].str.strip()

avg_price_per_neighborhood = data_neta.groupby("neighborhood")["price"].mean().reset_index()
avg_price_per_neighborhood.columns = ["nom_districte", "avg_price"]

barrios_gdf = barrios_gdf.merge(avg_price_per_neighborhood, on="nom_districte", how="left")

barcelona_map = folium.Map(location=[41.3851, 2.1734], zoom_start=12)

folium.Choropleth(
    geo_data=barrios_gdf,
    name="choropleth",
    data=barrios_gdf,
    columns=["nom_districte", "avg_price"],
    key_on="feature.properties.nom_districte",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Precio medio de vivienda por barrio (€)"
).add_to(barcelona_map)

folium.GeoJson(
    barrios_gdf,
    name="tooltip",
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'transparent'
    },
    tooltip=GeoJsonTooltip(
        fields=["nom_districte", "avg_price"],
        aliases=["Barrio", "Precio medio (€)"],
        localize=True,
        sticky=False,
        labels=True,
        max_width=300
    )
).add_to(barcelona_map)

In [ ]:
barrios_gdf

In [ ]:
barcelona_map